In [10]:
from hydra import initialize
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from lightning.pytorch.loggers import WandbLogger
import wandb
import sys
from dataio.MimicCXRDataset import CXRDataset
from torchvision import transforms
sys.path.append("..")
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from dataio.MimicCXRDataset import train_test_split_CXR
from models.ResnetMimic import ResnetMimic
with initialize(version_base=None, config_path="../configs"):
       cfg_training = compose(config_name="basic_resnet_experiment")
       cfg_dataset = compose(config_name="dataset_config")
print(cfg_training)
print(cfg_dataset)

{}
{'root_dir': '/Users/ago/PycharmProjects/mml/data/mimic-cxr-preprocessed', 'class_names': ['No Finding'], 'trans_resize': 224, 'view_position': 'PA', 'splitting': {'method': 'random', 'train_val_split': 0.6, 'test_val_split': 0.5}, 'seed': 42, 'uncertain_values': 'exclude'}


In [11]:
torch.__version__

'2.1.2'

In [12]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [20]:
# create datasets
train_dict, test_dict, val_dict = train_test_split_CXR(cfg_dataset)

transformList = [transforms.Resize(224),transforms.ToTensor()]

transform = transforms.Compose(transformList)

train_dataset = CXRDataset(label_name=cfg_dataset.class_names[0],labels_df=train_dict['labels'],images=train_dict['images'],transform=transform)

test_dataset = CXRDataset(label_name=cfg_dataset.class_names[0],labels_df=test_dict['labels'],images=test_dict['images'],transform=transform)

# create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)


Root dir:  /Users/ago/PycharmProjects/mml/data/mimic-cxr-preprocessed
Image matrix shape:  (43082, 224, 224)
Number of metadata rows:  43082
Dropping readings with uncertain values in selected class names
Number of filtered metadata rows:  43082
Number of patients in the training set:  25849
Number of patients in the val set:  8617
Number of patients in the test set:  8616
Number of images in train set:  25849
Number of images in val set:  8617
Number of images in test set:  8616


In [21]:
model = ResnetMimic()
wandb.finish()
if __name__ == '__main__':
    #init wandb_logger
    wandb.init()
    device='cpu'

    project_name = 'Resnet18_mimic'

    args = {
        'batch_size': 66,
        'learning_rate': 0.05,
        'project_name': project_name }

    wandb_logger = WandbLogger(project=project_name, log_model="all")
    wandb_logger.watch(model, log="all")

    trainer = pl.Trainer(max_epochs=3, logger=wandb_logger, accelerator='cpu')
    trainer.fit(model,train_dataloaders=train_dataloader,val_dataloaders=test_dataloader)
    wandb.finish()

/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name  | Type              | Params
--------------------------------------------
0 | model | ResNet            | 11.2 M
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.707    Total estima

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/ago/anaconda3/envs/mml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x512 and 224x1)

In [12]:
test_dataloader
